In [1]:
import torch
import torch.nn as nn
import sys
from pathlib import Path
sys.path.append(str(Path.cwd().parent))
from models.neuralnet import NeuralNet
from models.convnet import ConvNet

In [2]:
# Types of convolution available

from models.utils import return_conv_dict_keys
print(return_conv_dict_keys())
# How to create ConvNets
model = ConvNet(3, [['conv2d', '15,3,2'],
                    ['conv2dt', '35,3,2'],
                    ['conv2dt', '55,3,2'],
                    ['conv2d', '50,3,2']],
                     [[nn.MaxPool2d((2,2)),nn.BatchNorm2d(15),nn.ReLU()],
                      'skip', nn.ReLU(),
                      nn.ReLU()])
print(model(torch.randn(1,3,100,100)).shape)

model = ConvNet(3, 
                    [['conv2d', 'out_channels=15,kernel_size=3,stride=2'],
                    ['conv2dt', 'out_channels=30,kernel_size=3,stride=2'],
                    ['conv2dt', 'out_channels=50,kernel_size=3,stride=2'],
                    ['conv2d', 'out_channels=55,kernel_size=3,stride=2,padding=(2,2)']],
                    [[nn.MaxPool2d((2,2)),nn.BatchNorm2d(15),nn.ReLU()],
                     'skip', 
                     nn.ReLU(),
                     nn.ReLU()])
model(torch.randn(1,3,100,100)).shape == (1,55,51,51)

dict_keys(['conv1d', 'conv2d', 'conv3d', 'conv1dt', 'conv2dt', 'conv3dt'])
torch.Size([1, 50, 49, 49])


True

In [3]:
model = NeuralNet(100, [[15,True],
                        [20,False], 
                        2],
              [[torch.nn.ReLU(), torch.nn.BatchNorm1d(15)],
               [torch.nn.ReLU(), torch.nn.BatchNorm1d(20)],
               torch.nn.Sigmoid()])
model(torch.randn(4,100)).shape

torch.Size([4, 2])

Example for training ConvNet and NeuralNet to predict cat or dog  

Using ImageClass for image and test accuracy function, any function can be custom written on any dataset or task  

In [4]:
from models.utils import ImageClass,train,test_accuracy

In [7]:
import os
import numpy as np
dataset = os.listdir("train") # dataset files containing all files
np.random.seed(1)
np.random.shuffle(dataset)

In [8]:
dataset[:5]

['dog.6841.jpg',
 'cat.7288.jpg',
 'dog.2738.jpg',
 'dog.10090.jpg',
 'cat.6148.jpg']

In [9]:
label_list = [] # labeling the data for our case
for x in dataset:
    if 'cat' in x:
        label_list.append(0)
    else:
        label_list.append(1)

In [10]:
label_list[:5]

[1, 0, 1, 1, 0]

In [72]:
from sklearn.model_selection import train_test_split

Train test split, can be changed to any train, validation

In [73]:
x_train,x_test,y_train,y_test = train_test_split(dataset,label_list,random_state=1,test_size=0.3)

After creating dataset with labels and spliting for train and testing we are gonna define preprocessing steps to do on the data

In [74]:
from torchvision import transforms
torch.manual_seed(1)

In [75]:
# Preprocessing function for image data
preprocessing = transforms.Compose([transforms.ToPILImage(),
                                    transforms.Resize((250,250)),
                                    transforms.ToTensor(),
                                    transforms.Normalize((0.4,0.4,0.4),(0.1,0.1,0.1))
                                    ])

In [76]:
from torch.utils.data import DataLoader

# Params for ImageClass 
# x_train = list of images
# y_train = list of labels
# preprocessing = function for preprocessing of data after reading
# "train" -> folder of data

train_loader = DataLoader(ImageClass(x_train,y_train,preprocessing,"train"),batch_size=16)
test_loader = DataLoader(ImageClass(x_test,y_test,preprocessing,"train"),batch_size=1)

In [183]:
# First part for image extraction
model = ConvNet(3, [['conv2d', 'out_channels=50,kernel_size=15,stride=2'],
                    ['conv2d', 'out_channels=100,kernel_size=7,stride=2'],
                    ['conv2d', 'out_channels=150,kernel_size=3,stride=1'],
                    ['conv2d', 'out_channels=100,kernel_size=3,stride=1']],
                    [[nn.MaxPool2d((2,3)),nn.BatchNorm2d(50),nn.ReLU()],
                     [nn.MaxPool2d((2,2)),nn.BatchNorm2d(100),nn.ReLU()], 
                     [nn.MaxPool2d((2,2)),nn.BatchNorm2d(150),nn.ReLU()],
                     [nn.ReLU(),nn.Flatten()]])

with torch.no_grad():
    model.eval()
    out_shape = model(torch.randn((1,3,250,250),requires_grad=False)).shape[1]
    model.train()
print(out_shape)
# NeuralNet for feature classification extracted by the first model
model2 = NeuralNet(out_shape, [528,500,9],
              [torch.nn.ReLU(),torch.nn.ReLU(),"skip"])

# We put ConvNet and Neural net in a list to pass to our custom model which combines both of them
# Making it the final model
models = [model,model2]


300


In [184]:
from models.custom import CustomModel

In [185]:
# output step for neural net output
def prediction_preprocess(x):
    return torch.softmax(x,dim=1)

In [186]:
final_model = CustomModel(models,prediction_preprocess).cuda() 
optim = torch.optim.Adam(final_model.parameters(),lr=1e-4) #optimizer
scheduler = torch.optim.lr_scheduler.StepLR(optim,step_size=12,gamma=0.1) #lr_scheduler 
loss = nn.CrossEntropyLoss() #classification loss

In [187]:
# Test function written for our own need
def test(epoch,final_model,test_loader):
    correct = 0 
    final_model.eval()
    with torch.no_grad():
        for x,y in test_loader:
            output = final_model(x.cuda())
            correct+=(torch.argmax(output,dim=1) == y.cuda()).item()
        print("Epoch {} - Accuracy - {:.2f}%".format(epoch,(correct /len(test_loader))*100))

In [188]:
for epochs in range(30):
    train(epochs,final_model,optim,train_loader,loss)
    scheduler.step()
    test(epochs,final_model,test_loader)

Train Epoch: 0 Length 35 	Loss: 2.193600
Epoch 0 - Accuracy - 21.19%
Train Epoch: 1 Length 35 	Loss: 2.136061
Epoch 1 - Accuracy - 33.90%
Train Epoch: 2 Length 35 	Loss: 1.839989
Epoch 2 - Accuracy - 37.29%
Train Epoch: 3 Length 35 	Loss: 1.778563
Epoch 3 - Accuracy - 44.92%
Train Epoch: 4 Length 35 	Loss: 2.069886
Epoch 4 - Accuracy - 54.24%
Train Epoch: 5 Length 35 	Loss: 1.498630
Epoch 5 - Accuracy - 56.36%
Train Epoch: 6 Length 35 	Loss: 1.712986
Epoch 6 - Accuracy - 59.75%
Train Epoch: 7 Length 35 	Loss: 1.536920
Epoch 7 - Accuracy - 60.59%
Train Epoch: 8 Length 35 	Loss: 1.398962
Epoch 8 - Accuracy - 63.56%
Train Epoch: 9 Length 35 	Loss: 1.427579
Epoch 9 - Accuracy - 61.86%
Train Epoch: 10 Length 35 	Loss: 1.550839
Epoch 10 - Accuracy - 65.25%
Train Epoch: 11 Length 35 	Loss: 1.376087
Epoch 11 - Accuracy - 65.25%
Train Epoch: 12 Length 35 	Loss: 1.378455
Epoch 12 - Accuracy - 66.10%
Train Epoch: 13 Length 35 	Loss: 1.450982
Epoch 13 - Accuracy - 66.95%
Train Epoch: 14 Length 35 